In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg, european_call_payoff

## Tests

In [2]:
heston_params = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3)
state         = MarketState(stock_price = 100., interest_rate = 0.)
model         = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)

r_x           = np.load(r"Data/anderson tg/r_x.npy")
f_nu_y        = np.load(r"Data/anderson tg/f_nu_y.npy")
f_sigma_y     = np.load(r"Data/anderson tg/f_sigma_y.npy")
kwargs        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

### At the money

In [3]:
strike = 100.
T = 1.
ec_payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 5e-2, "batch_size": 100_000, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": True}

In [4]:
model.call_price(T, strike)

16.69960076395676

In [5]:
mc_price(N_T = 100, simulate = simulate_heston_euler, **common_mc_params)

Random seed:                42
Number of simulate calls:   27
MAX_ITER:                   100000
Number of paths:            5400000
Absolute error:             0.05
Length of the conf intl:    0.04972889484394921
Confidence level:           0.05



16.67879440437618

In [6]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_qe, **common_mc_params)

Random seed:                42
Number of simulate calls:   27
MAX_ITER:                   100000
Number of paths:            5400000
Absolute error:             0.05
Length of the conf intl:    0.049283458891177016
Confidence level:           0.05



16.66785637963097

In [7]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_tg, **common_mc_params, **kwargs)

Random seed:                42
Number of simulate calls:   27
MAX_ITER:                   100000
Number of paths:            5400000
Absolute error:             0.05
Length of the conf intl:    0.04932953799894217
Confidence level:           0.05



16.6923226133552

In [ ]:
for scheme in [simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg]:
    print(scheme.__name__, "\n")
    for bs in range(50_000, 150_000, 10_000):
        print(bs)
        %timeit -r1 mc_price(N_T = 50, absolute_error=5e-2, verbose = False, state = state, payoff = payoff, simulate = scheme, batch_size=bs, heston_params = heston_params, T = T)

    print("\n\n\n\n")

### Out of the money

In [8]:
strike = 10.
T = 1.
ec_payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 5e-2, "batch_size": 100_000, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": True}

In [9]:
model.call_price(T, strike)

90.00070826494877

In [10]:
mc_price(N_T = 100, simulate = simulate_heston_euler, **common_mc_params)

Random seed:                42
Number of simulate calls:   58
MAX_ITER:                   100000
Number of paths:            11600000
Absolute error:             0.05
Length of the conf intl:    0.049793188506487104
Confidence level:           0.05



90.0043906779468

In [11]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_qe, **common_mc_params)

Random seed:                42
Number of simulate calls:   57
MAX_ITER:                   100000
Number of paths:            11400000
Absolute error:             0.05
Length of the conf intl:    0.04984236677321091
Confidence level:           0.05



90.19859406126821

In [12]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_tg, **common_mc_params, **kwargs)

Random seed:                42
Number of simulate calls:   57
MAX_ITER:                   100000
Number of paths:            11400000
Absolute error:             0.05
Length of the conf intl:    0.04987432685300773
Confidence level:           0.05



90.20163788565202